# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
#Install the dependencies

In [4]:
%%capture
!pip install dlt[duckdb]

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**

In [5]:
# to do: homework :)
limit = 5
generator = square_root_generator(limit)

q1_answer = sum(generator)
print(q1_answer)


8.382332347441762


- **Question 2: What is the 13th number yielded**

In [6]:
limit = 13
generator = square_root_generator(limit)

for i in range(1, limit):
    next(generator)
q2_answer = next(generator)
print(q2_answer)


3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [7]:
import duckdb
import dlt

table_name="users"
query = f"select sum(age) from {table_name}"

# Load the first generator and calculate the sum of ages of all people.
# Make sure to only load it once.
pipeline = dlt.pipeline(
    pipeline_name="quick_start",
    destination="duckdb",
    dataset_name="mydata"
)
pipeline.run(
    people_1,
    table_name=table_name,
    write_disposition="replace"
)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
display("age sum of first generator:", conn.sql(query))

# Append the second generator to the same table as the first.
load_info = pipeline.run(
    people_2,
    table_name=table_name,
    write_disposition="append"
)
df = conn.sql(f"SELECT * FROM {table_name}").df()
display(df)

# After correctly appending the data, calculate the sum of all ages of people.
q2_answer = conn.sql(query).fetchone()[0]
display("age sum of second generator appended to first:", conn.sql(query))
print(f"Question #2 Answer: {q2_answer}")

# Bonus section related to Question #3
print("\nBonus: Question 3 Answer")
load_info = pipeline.run(
    people_2,
    table_name=table_name,
    write_disposition="merge"
)

display("Question #3 Answer:", conn.sql(query))


'age sum of first generator:'

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1706780087.111512,Qo7OzYJsxcPlNQ,None
1,2,Person_2,27,City_A,1706780087.111512,w/RG0mz09vDEuQ,None
2,3,Person_3,28,City_A,1706780087.111512,AAwtTXoCGV8Z9g,None
3,4,Person_4,29,City_A,1706780087.111512,IsSBvrDBR8cB+Q,None
4,5,Person_5,30,City_A,1706780087.111512,2LwJqqDvI4TZNw,None
5,3,Person_3,33,City_B,1706780088.1842322,lmqZsnKPdS8XZA,Job_3
6,4,Person_4,34,City_B,1706780088.1842322,KSM3SYVjdwMFLg,Job_4
7,5,Person_5,35,City_B,1706780088.1842322,bwkYseO7eO74MA,Job_5
8,6,Person_6,36,City_B,1706780088.1842322,GMhnQukeYDKlBQ,Job_6
9,7,Person_7,37,City_B,1706780088.1842322,j5xYw9olVtNeIg,Job_7


'age sum of second generator appended to first:'

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

Question #2 Answer: 353

Bonus: Question 3 Answer


'Question #3 Answer:'

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      213 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [8]:
import duckdb
import dlt

table_name = "people_new"

pipeline = dlt.pipeline(
    destination="duckdb",
    dataset_name="people_merged",
)
pipeline.run(
    people_1,
    table_name=table_name,
    write_disposition="replace",
    primary_key="ID"
)
pipeline.run(
    people_2,
    table_name=table_name,
    write_disposition="merge"
)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# After loading, you should have a total of 8 records,
df = conn.sql(f"SELECT * FROM {table_name}").df()
display(df)
assert df.shape[0] == 8

# and ID 3 should have age 33.
person_id3_age = df.loc[df["id"] == 3, "age"].values[0]
assert int(person_id3_age) == 33

q3_answer = df[df["occupation"].notna()]["age"].sum()
display("Question #3 Answer:", q3_answer)


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1706780090.390226,31a7ZjPxh15mWg,None
1,2,Person_2,27,City_A,1706780090.390226,ixfpW/qyVck3aA,None
2,5,Person_5,35,City_B,1706780091.4627125,Da8a60wN95i8WA,Job_5
3,4,Person_4,34,City_B,1706780091.4627125,Hmv7oLezQTURdg,Job_4
4,8,Person_8,38,City_B,1706780091.4627125,lE5yrnmYKrP9TQ,Job_8
5,7,Person_7,37,City_B,1706780091.4627125,PbNqIjLpwP0kyQ,Job_7
6,3,Person_3,33,City_B,1706780091.4627125,88YQGkzisJQthQ,Job_3
7,6,Person_6,36,City_B,1706780091.4627125,OFeQ7RgEx2U1ww,Job_6


'Question #3 Answer:'

213

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX